AI Engineering Bootcamp Cohort 4 Midterm

#### Install our key components for RAG etc

In [1]:
!pip install langchain-core==0.2.27 langchain-community==0.2.10
!pip install langchain-experimental==0.0.64 langgraph-checkpoint==1.0.6 langgraph==0.2.16 langchain-qdrant==0.1.3
!pip install langchain-openai==0.1.9

#### Install our vector store - Qdrant

In [2]:
!pip install -qU qdrant-client

: 

#### Install supporting utilities

In [3]:
!pip install -qU tiktoken pymupdf
from langchain_community.document_loaders import PyMuPDFLoader

Environment Variables

- get OpenAI API Key - will use some of the OpenAI models

In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

Number of pages in 1: 73
Number of pages in 2: 64
Title of Document 1: Blueprint for an AI Bill of Rights
Title of Document 2: Artificial Intelligence Risk Management Framework: Generative Artificial Intelligence Profile
Full metadata for Document 1: {'format': 'PDF 1.6', 'title': 'Blueprint for an AI Bill of Rights', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe Illustrator 26.3 (Macintosh)', 'producer': 'iLovePDF', 'creationDate': "D:20220920133035-04'00'", 'modDate': "D:20221003104118-04'00'", 'trapped': '', 'encryption': None}
Full metadata for Document 2: {'format': 'PDF 1.6', 'title': 'Artificial Intelligence Risk Management Framework: Generative Artificial Intelligence Profile', 'author': 'National Institute of Standards and Technology', 'subject': '', 'keywords': '', 'creator': 'Acrobat PDFMaker 24 for Word', 'producer': 'Adobe PDF Library 24.2.159', 'creationDate': "D:20240805141702-04'00'", 'modDate': "D:20240805143048-04'00'", 'trapped': '', 'encryption': No

In [12]:
from utilities.rag_utilities import create_vector_store

qdrant_retriever = create_vector_store()

Number of pages in 1: 73
Number of pages in 2: 64
Title of Document 1: Blueprint for an AI Bill of Rights
Title of Document 2: Artificial Intelligence Risk Management Framework: Generative Artificial Intelligence Profile
Full metadata for Document 1: {'format': 'PDF 1.6', 'title': 'Blueprint for an AI Bill of Rights', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe Illustrator 26.3 (Macintosh)', 'producer': 'iLovePDF', 'creationDate': "D:20220920133035-04'00'", 'modDate': "D:20221003104118-04'00'", 'trapped': '', 'encryption': None}
Full metadata for Document 2: {'format': 'PDF 1.6', 'title': 'Artificial Intelligence Risk Management Framework: Generative Artificial Intelligence Profile', 'author': 'National Institute of Standards and Technology', 'subject': '', 'keywords': '', 'creator': 'Acrobat PDFMaker 24 for Word', 'producer': 'Adobe PDF Library 24.2.159', 'creationDate': "D:20240805141702-04'00'", 'modDate': "D:20240805143048-04'00'", 'trapped': '', 'encryption': No

Test it

In [13]:
query = "How should you be protected from abusive data practices "
results = qdrant_retriever.get_relevant_documents(query)

for result in results:
    print(result.page_content)
    print(result.metadata)
    print("---")



You should be protected from abusive data practices via built-in 
protections and you should have agency over how data about 
you is used. You should be protected from violations of privacy through 
design choices that ensure such protections are included by default, including 
ensuring that data collection conforms to reasonable expectations and that 
only data strictly necessary for the specific context is collected. Designers, de­
velopers, and deployers of automated systems should seek your permission 
and respect your decisions regarding collection, use, access, transfer, and de­
letion of your data in appropriate ways and to the greatest extent possible; 
where not possible, alternative privacy by design safeguards should be used. 
Systems should not employ user experience and design decisions that obfus­
cate user choice or burden users with defaults that are privacy invasive. Con­
sent should only be used to justify collection of data in cases where it can be 
appropriately and

/home/rchrdgwr/anaconda3/envs/llmops-course/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [53]:
query = "tell me about Karen Hao"
results = qdrant_retriever.get_relevant_documents(query)

for result in results:
    print(result.page_content)
    print(result.metadata)
    print("---")

ENDNOTES
75. See., e.g., Sam Sabin. Digital surveillance in a post-Roe world. Politico. May 5, 2022. https://
www.politico.com/newsletters/digital-future-daily/2022/05/05/digital-surveillance-in-a-post-roe­
world-00030459; Federal Trade Commission. FTC Sues Kochava for Selling Data that Tracks People at
Reproductive Health Clinics, Places of Worship, and Other Sensitive Locations. Aug. 29, 2022. https://
www.ftc.gov/news-events/news/press-releases/2022/08/ftc-sues-kochava-selling-data-tracks-people­
reproductive-health-clinics-places-worship-other
76. Todd Feathers. This Private Equity Firm Is Amassing Companies That Collect Data on America’s
Children. The Markup. Jan. 11, 2022.
https://themarkup.org/machine-learning/2022/01/11/this-private-equity-firm-is-amassing-companies­
that-collect-data-on-americas-children
77. Reed Albergotti. Every employee who leaves Apple becomes an ‘associate’: In job databases used by
employers to verify resume information, every former Apple employee’s tit

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
system_template = """
    You are an expert at explaining technical documents to people.
    You are provided context below to answer the question.
    Only use the information provided below.
    If you cannot answer the question with the content below say 'I don't have enough information, sorry'
    The two documents are 'Blueprint for an AI Bill of Rights' and 'Artificial Intelligence Risk Management Framework: Generative Artificial Intelligence Profile'
"""
human_template = """ 
===
question:
{question}

===
context:
{context}
===
"""
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])
# create the chain
openai_chat_model = ChatOpenAI(model="gpt-4o")
chain = chat_prompt | openai_chat_model
print(chain.invoke({"question": "Can you give me a summary of the 2 documents", "context":""})) # displays 


content="I don't have enough information, sorry." response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 112, 'total_tokens': 120, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_52a7f40b0b', 'finish_reason': 'stop', 'logprobs': None} id='run-511e9c6e-8c54-4ca3-8991-18bb3ba7c744-0' usage_metadata={'input_tokens': 112, 'output_tokens': 8, 'total_tokens': 120}


In [21]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))


    | {"response": chat_prompt | openai_chat_model, "context": itemgetter("context")}
)
response = retrieval_augmented_qa_chain.invoke({"question" : "What is the AI Bill of Rights "})
print(response)
print(response["response"].content)
print(f"Number of found context: {len(response['context'])}")

{'response': AIMessage(content='The AI Bill of Rights, officially known as the "Blueprint for an AI Bill of Rights," is a set of five principles and associated practices designed to guide the design, use, and deployment of automated systems. Its primary goal is to protect the rights of the American public in the age of artificial intelligence (AI). Developed through extensive consultation with various stakeholders, including impacted communities, industry stakeholders, technology developers, and policymakers, these principles are intended to ensure that AI systems are aligned with democratic values and protect civil rights, civil liberties, and privacy. The Blueprint provides a national values statement and a toolkit that is sector-agnostic, meaning it can be applied across different sectors to inform policy decisions and the technological design process.', response_metadata={'token_usage': {'completion_tokens': 144, 'prompt_tokens': 1848, 'total_tokens': 1992, 'completion_tokens_detai